In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 100     # number of variables
num_ineq = 100    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msLinear
model = msLinear(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    if sum(model.cal_violation()) > 0:
        xval, objval = model.solve("gurobi", tee=True)
        break
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_exact_100-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:47:05<00:00, 64.26s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -81.422441               0.0     64.195420
std      0.467911               0.0     34.932151
min    -82.461194               0.0     60.602567
25%    -81.744373               0.0     60.650923
50%    -81.426739               0.0     60.686694
75%    -81.098760               0.0     60.737455
max    -80.281595               0.0    410.022610
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_rnd_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.56it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -82.607990          3.890467      0.171419
std      0.478083          0.863671      0.014108
min    -83.612986          1.969477      0.153313
25%    -82.917768          3.326827      0.162927
50%    -82.692222          3.821674      0.167191
75%    -82.300442          4.492946      0.175870
max    -81.354785          6.151293      0.235932
Number of infeasible solution: 100


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_n1_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:06<00:00,  1.51it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     7.424101               0.0      0.625923
std     14.755239               0.0      0.484593
min    -46.777448               0.0      0.206343
25%     -1.315255               0.0      0.339572
50%      8.883279               0.0      0.400709
75%     15.944699               0.0      0.651099
max     46.067285               0.0      2.322423
Number of infeasible solution: 0


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 3603.47
Epoch 1, Validation Loss: -34.79
Epoch 2, Validation Loss: -33.52
Epoch 3, Validation Loss: -4.05
Epoch 4, Validation Loss: -47.93
Epoch 5, Validation Loss: -29.48
Epoch 6, Validation Loss: 115.89
Epoch 7, Validation Loss: -50.66
Epoch 8, Validation Loss: -46.35
Epoch 9, Validation Loss: -35.02
Epoch 10, Validation Loss: -51.17
Epoch 11, Validation Loss: -53.58
Epoch 12, Validation Loss: -39.70
Epoch 13, Validation Loss: -55.25
Epoch 14, Validation Loss: -55.02
Epoch 15, Validation Loss: -42.40
Epoch 16, Validation Loss: -23.25
Epoch 17, Validation Loss: -53.65
Epoch 18, Validation Loss: -55.95
Epoch 19, Validation Loss: -58.33
Epoch 20, Validation Loss: -49.98
Epoch 21, Validation Loss: -57.68
Epoch 22, Validation Loss: -61.21
Epoch 23, Validation Loss: -58.09
Epoch 24, Validation Loss: -60.00
Epoch 25, Validation Loss: -61.61
Epoch 26, Validation Loss: -63.04
Epoch 27, Validation Loss: -64.91
Epoch 28, Validation Loss: -65.75
Epoch 29, Validation Los

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lr_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 27.87it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -71.006230          0.000957      0.003207
std      0.651362          0.009574      0.000890
min    -71.615804          0.000000      0.001983
25%    -71.248249          0.000000      0.002539
50%    -71.226797          0.000000      0.003003
75%    -71.182536          0.000000      0.003525
max    -68.618704          0.095736      0.006512
Number of infeasible solution: 1


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2485.01
Epoch 1, Validation Loss: -32.15
Epoch 2, Validation Loss: 0.04
Epoch 3, Validation Loss: -32.32
Epoch 4, Validation Loss: -46.01
Epoch 5, Validation Loss: 42.80
Epoch 6, Validation Loss: -47.26
Epoch 7, Validation Loss: -8.09
Epoch 8, Validation Loss: 77.56
Epoch 9, Validation Loss: -34.67
Epoch 10, Validation Loss: -55.31
Epoch 11, Validation Loss: -54.18
Epoch 12, Validation Loss: -49.20
Epoch 13, Validation Loss: -56.36
Epoch 14, Validation Loss: -55.54
Epoch 15, Validation Loss: -50.56
Epoch 16, Validation Loss: -58.26
Epoch 17, Validation Loss: -16.20
Epoch 18, Validation Loss: -56.60
Epoch 19, Validation Loss: -60.42
Epoch 20, Validation Loss: -57.51
Epoch 21, Validation Loss: -58.88
Epoch 22, Validation Loss: -47.80
Epoch 23, Validation Loss: -50.84
Epoch 24, Validation Loss: -64.70
Epoch 25, Validation Loss: -44.12
Epoch 26, Validation Loss: -63.51
Epoch 27, Validation Loss: -65.26
Epoch 28, Validation Loss: -64.06
Epoch 29, Validation Loss: -

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lt_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.62it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -71.518873               0.0      0.003425
std      0.163475               0.0      0.001103
min    -71.772086               0.0      0.001989
25%    -71.585936               0.0      0.002601
50%    -71.519976               0.0      0.003004
75%    -71.484812               0.0      0.004003
max    -70.257026               0.0      0.006513
Number of infeasible solution: 0


### Parametric Learning Then Rounding

In [22]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [23]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [24]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmLinear(["b", "x"], num_var, num_ineq, penalty_weight)

In [25]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2482.40
Epoch 1, Validation Loss: -28.92
Epoch 2, Validation Loss: -45.84
Epoch 3, Validation Loss: -46.05
Epoch 4, Validation Loss: -48.96
Epoch 5, Validation Loss: -49.19
Epoch 6, Validation Loss: -47.22
Epoch 7, Validation Loss: -48.44
Epoch 8, Validation Loss: -54.44
Epoch 9, Validation Loss: -49.65
Epoch 10, Validation Loss: -36.42
Epoch 11, Validation Loss: -51.96
Epoch 12, Validation Loss: 11.63
Epoch 13, Validation Loss: -56.89
Epoch 14, Validation Loss: -54.86
Epoch 15, Validation Loss: -54.65
Epoch 16, Validation Loss: -60.34
Epoch 17, Validation Loss: -57.01
Epoch 18, Validation Loss: -18.64
Epoch 19, Validation Loss: -57.13
Epoch 20, Validation Loss: -58.64
Epoch 21, Validation Loss: -58.42
Epoch 22, Validation Loss: -59.11
Epoch 23, Validation Loss: -60.23
Epoch 24, Validation Loss: -63.98
Epoch 25, Validation Loss: -63.48
Epoch 26, Validation Loss: -9.25
Epoch 27, Validation Loss: -63.98
Epoch 28, Validation Loss: -37.10
Epoch 29, Validation Loss

In [26]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_pr_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.46it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean  -7.439199e+01          0.037471      0.000665
std    7.141223e-14          0.070150      0.000599
min   -7.439199e+01          0.000000      0.000000
25%   -7.439199e+01          0.000000      0.000000
50%   -7.439199e+01          0.000000      0.000999
75%   -7.439199e+01          0.049921      0.001003
max   -7.439199e+01          0.358276      0.002051
Number of infeasible solution: 42
